In [1]:
import pandas as pd
import numpy as np

In [2]:
def json_to_df(json_file):
  arrayForDF = []
  for current_subject in json_file['data']:
    subject=current_subject['title']
    for current_context in current_subject['paragraphs']:
      context = current_context['context']
      for current_question in current_context['qas']:
        question = current_question['question']
        for answer in current_question["answers"]:
          answer_text=answer['text']
          answer_start= answer['answer_start']
          record = {
                  "answer_text": answer_text,
                  'answer_start':answer_start,
                  "question":question,
                  "context": context,
                  'subject':subject
              }
          arrayForDF.append(record)
  df=pd.DataFrame(arrayForDF)
  return df

In [3]:
df1=pd.read_json('train-v1.1.json')
df2=pd.read_json('dev-v1.1.json')

In [4]:
train_df1=json_to_df(df1)
#test_df=json_to_df(df2)

In [5]:
train_df=train_df1.head(50000)
test_df=train_df1.iloc[50001:]

In [6]:
test_df.reset_index(inplace=True)
del test_df['index']
test_df

,answer_text,answer_start,question,context,subject
0,Australian citizenship,504,What was a necessary qualification for voting ...,Norfolk Island was originally a colony acquire...,Norfolk_Island
1,immigration,103,What has been heavily restricted in Norfolk Is...,The island is subject to separate immigration ...,Norfolk_Island
2,2012,194,When were the immigration protocols relaxed in...,The island is subject to separate immigration ...,Norfolk_Island
3,an Unrestricted Entry Permit for all Australia...,259,What aided in the relaxation of immigration pr...,The island is subject to separate immigration ...,Norfolk_Island
4,pass a police check and be able to pay into th...,409,What must Australian and New Zealand citizens ...,The island is subject to separate immigration ...,Norfolk_Island
...,...,...,...,...,...
37593,Oregon,229,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...",Kathmandu
37594,Rangoon,414,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...",Kathmandu
37595,Minsk,476,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...",Kathmandu
37596,1975,199,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...",Kathmandu


In [7]:
from gensim.models import Word2Vec
from gensim.models import FastText
import gensim

#Loading contexts
contexts= []

for context in train_df.context.unique():
  contexts.append(context)

for context in test_df.context.unique():
  contexts.append(context)

contexts_df=pd.DataFrame(contexts)

#Context preprocessing
contexts_df[0]=contexts_df[0].apply(lambda x: gensim.utils.simple_preprocess(x))

#Training our wordembeding models
contexts_model=Word2Vec(contexts_df[0].to_list(), vector_size=300,workers = -1)
google_model= gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/nlp/GoogleNews-vectors-negative300.bin', binary=True)
festTextModel=FastText(contexts_df[0].to_list(),vector_size=300, workers=-1)

In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

def get_answer_context(df):
  Length_context=0
  answer=""
  for sentence in sent_tokenize(df.context):
    Length_context+=len(sentence)+1
    if df.answer_start<=Length_context:
      if len(sentence)>=len(str(df.answer_text)):
        if answer=="":
          return sentence
        else:
          return answer+" "+sentence
      else:
        answer+=sentence

train_df['entire_answer_text']= train_df.apply(lambda row: get_answer_context(row), axis=1)
train_df.dropna(subset=['entire_answer_text'],inplace=True)
train_df['entire_answer_text']= train_df['entire_answer_text'].apply(lambda x: gensim.utils.simple_preprocess(x))
train_df.drop(train_df[train_df.astype(str)['entire_answer_text']=='[]'].index, inplace=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-8-66ad9eefe98f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['entire_answer_text']= train_df.apply(lambda row: get_answer_context(row), axis=1)
<ipython-input-8-66ad9eefe98f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset=['entire_answer_text'],inplace=True)
<ipython-input-8-66ad9eefe98f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [9]:
train_df['question_better_preprocess']= train_df['question'].apply(lambda x: gensim.utils.simple_preprocess(x))
train_df.drop(train_df[train_df.astype(str)['question_better_preprocess']=='[]'].index, inplace=True)

<ipython-input-9-a9937457c8bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['question_better_preprocess']= train_df['question'].apply(lambda x: gensim.utils.simple_preprocess(x))
<ipython-input-9-a9937457c8bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(train_df[train_df.astype(str)['question_better_preprocess']=='[]'].index, inplace=True)


In [10]:
def get_vectors_word2vec(sentence):
  return contexts_model[sentence]

def get_vectors_fasttext(sentence):
  return festTextModel.wv[sentence]

in_voc=0
not_in_voc= 0
in_google_voc=0

def get_vectors_mix(sentence):
  global in_voc
  global not_in_voc
  global in_google_voc
  #print("Word2Vec(Context):"+ str(in_voc) +'/'+ "Word2Vec(Google)" + str(in_google_voc) + "FastText:"+str(not_in_voc), end='\r')
  sentence_word2_vec = 1
  sentence_google_word2vec = 1
  for word in sentence:
    if(word not in contexts_model.wv.key_to_index):
      sentence_word2_vec = 0
    if(word not in google_model.key_to_index):
      sentence_google_word2vec = 0
  if(sentence_word2_vec == 1):
    in_voc +=1
    return contexts_model.wv[sentence]
  elif(sentence_google_word2vec == 1):
    in_google_voc +=1
    return google_model[sentence]
  else:
    not_in_voc += 1
    return festTextModel.wv[sentence]

In [11]:
train_df['question_vectors_fasttext']=train_df['question_better_preprocess'].apply(get_vectors_fasttext)
train_df['question_vectors_fasttext_sentence']=train_df['question_vectors_fasttext'].apply(lambda a: np.mean(a,axis=0))

<ipython-input-11-2bf0eb9ec025>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['question_vectors_fasttext']=train_df['question_better_preprocess'].apply(get_vectors_fasttext)
<ipython-input-11-2bf0eb9ec025>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['question_vectors_fasttext_sentence']=train_df['question_vectors_fasttext'].apply(lambda a: np.mean(a,axis=0))


In [12]:
train_df['entire_answer_fasttext_vectors']=train_df['entire_answer_text'].apply(get_vectors_fasttext)
train_df['entire_answer_fasttext_vectors_sentence']=train_df['entire_answer_fasttext_vectors'].apply(lambda a:np.mean(a, axis=0))

<ipython-input-12-0b2a14f77c25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['entire_answer_fasttext_vectors']=train_df['entire_answer_text'].apply(get_vectors_fasttext)
<ipython-input-12-0b2a14f77c25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['entire_answer_fasttext_vectors_sentence']=train_df['entire_answer_fasttext_vectors'].apply(lambda a:np.mean(a, axis=0))


In [13]:
def to_processable_array(series):
  df=pd.DataFrame()
  for i in range(len(series)):
    df0=pd.DataFrame(series.iloc[i])
    df0.dropna(inplace=True)
    df=df.append(df0.T)
  df.reset_index(inplace=True)
  df.T
  del df['index']
  return df

In [14]:
question_fasttext_np=to_processable_array(train_df['question_vectors_fasttext_sentence'])
answer_fasttext_np=to_processable_array(train_df['entire_answer_fasttext_vectors_sentence'])

Streaming output truncated to the last 5000 lines.
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)

In [15]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

regressor = SVR(gamma= 'auto')
SVR_FastText = MultiOutputRegressor(regressor, n_jobs=1)
SVR_FastText.fit(question_fasttext_np, answer_fasttext_np)


MultiOutputRegressor(estimator=SVR(gamma='auto'), n_jobs=1)

In [ ]:
# preparing testing dataset
test_df['entire_answer_text']= test_df.apply(lambda row: get_answer_context(row), axis=1)
test_df.dropna(subset=['entire_answer_text'],inplace=True)
test_df['entire_answer_text']= test_df['entire_answer_text'].apply(lambda x: gensim.utils.simple_preprocess(x))
test_df.drop(test_df[test_df.astype(str)['entire_answer_text']=='[]'].index, inplace=True)

test_df['question_better_preprocess']= test_df['question'].apply(lambda x: gensim.utils.simple_preprocess(x))
test_df.drop(test_df[test_df.astype(str)['question_better_preprocess']=='[]'].index, inplace=True)

test_df['question_vectors_fasttext']=test_df['question_better_preprocess'].apply(get_vectors_fasttext)
test_df['question_vectors_fasttext_sentence']=test_df['question_vectors_fasttext'].apply(lambda a: np.mean(a,axis=0))

test_df['entire_answer_fasttext_vectors']=test_df['entire_answer_text'].apply(get_vectors_fasttext)
test_df['entire_answer_fasttext_vectors_sentence']=test_df['entire_answer_fasttext_vectors'].apply(lambda a:np.mean(a, axis=0))

question_fasttext_np_test=to_processable_array(test_df['question_vectors_fasttext_sentence'])
answer_fasttext_np=to_processable_array(test_df['entire_answer_fasttext_vectors_sentence'])

Streaming output truncated to the last 5000 lines.
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)
<ipython-input-13-f471a5c00ed7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df0.T)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

def get_vectorized_context(context):
  context_vectors=[]
  for sent in sent_tokenize(context):
    #In case you are using word2vec, we need the mean of all words in a sentence
    sent_vectors=[]
    sent_vectors.append(get_vectors_mix(sent))
    context_vectors.append({
        'vector': np.array(sent_vectors).mean(axis=0),
        'text': gensim.utils.simple_preprocess(sent)
    })
  return context_vectors

def least_distance(predicted, context_sentence_vector):
  least_distance_test=""
  least_distance = 100
  for sentence in context_sentence_vector:
    current_distance= euclidean_distances(predicted, [sentence['vector']])
    if current_distance<least_distance:
      least_distance_text = sentence['text']
      least_distance= current_distance
  return least_distance_text

In [19]:
# For SVR_FastText model
scores={'correct':0, 'incorrect':0}
last_context=""
for index, row in test_df.iterrows():
  if row['context'] != last_context:
    last_context = row['context']
    context_vector = get_vectorized_context(row['context'])
  answer_predicted = SVR_FastText.predict([question_fasttext_np_test.iloc[index]])
  best_text_match_from_context = least_distance(answer_predicted, context_vector)
  if best_text_match_from_context ==row['entire_answer_text']:
    scores['correct'] += 1
  else:
    scores['incorrect'] += 1
print(scores)
print(scores['correct']/test_df.shape[0])

IndexError: ignored

In [ ]:
train_df

In [21]:
test_df

,answer_text,answer_start,question,context,subject,entire_answer_text,question_better_preprocess,question_vectors_fasttext,question_vectors_fasttext_sentence,entire_answer_fasttext_vectors,entire_answer_fasttext_vectors_sentence
0,Australian citizenship,504,What was a necessary qualification for voting ...,Norfolk Island was originally a colony acquire...,Norfolk_Island,"[again, in, the, high, court, of, australia, a...","[what, was, necessary, qualification, for, vot...","[[-0.0004852763, -5.1644292e-05, -8.1352315e-0...","[1.0463513e-05, -0.00033290745, -0.0001997083,...","[[0.00039981573, -0.00016550168, -0.001193517,...","[-0.00013445973, 9.919431e-05, -0.00020630483,..."
1,immigration,103,What has been heavily restricted in Norfolk Is...,The island is subject to separate immigration ...,Norfolk_Island,"[until, recently, immigration, to, norfolk, is...","[what, has, been, heavily, restricted, in, nor...","[[-0.0004852763, -5.1644292e-05, -8.1352315e-0...","[-5.8543366e-05, -0.00031481765, 4.9198035e-05...","[[-0.00017197868, -5.440683e-05, 0.0011636026,...","[-0.00014885343, 3.3016768e-06, 0.0002792439, ..."
2,2012,194,When were the immigration protocols relaxed in...,The island is subject to separate immigration ...,Norfolk_Island,"[in, immigration, controls, were, relaxed, wit...","[when, were, the, immigration, protocols, rela...","[[-2.615275e-05, 0.0003023257, 0.00043555113, ...","[-4.2833076e-06, -0.0002038915, 5.7476398e-05,...","[[0.00080009917, -0.0023480232, -0.0011538627,...","[0.000118224234, -2.8048446e-05, -0.0001101853..."
3,an Unrestricted Entry Permit for all Australia...,259,What aided in the relaxation of immigration pr...,The island is subject to separate immigration ...,Norfolk_Island,"[in, immigration, controls, were, relaxed, wit...","[what, aided, in, the, relaxation, of, immigra...","[[-0.0004852763, -5.1644292e-05, -8.1352315e-0...","[-0.00019071928, -7.7108045e-05, -2.9343062e-0...","[[0.00080009917, -0.0023480232, -0.0011538627,...","[0.000118224234, -2.8048446e-05, -0.0001101853..."
4,pass a police check and be able to pay into th...,409,What must Australian and New Zealand citizens ...,The island is subject to separate immigration ...,Norfolk_Island,"[in, immigration, controls, were, relaxed, wit...","[what, must, australian, and, new, zealand, ci...","[[-0.0004852763, -5.1644292e-05, -8.1352315e-0...","[0.00016168565, -0.00017584911, -0.00026230508...","[[0.00080009917, -0.0023480232, -0.0011538627,...","[0.000118224234, -2.8048446e-05, -0.0001101853..."
...,...,...,...,...,...,...,...,...,...,...,...
37593,Oregon,229,In what US state did Kathmandu first establish...,"Kathmandu Metropolitan City (KMC), in order to...",Kathmandu,"[kmc, first, international, relationship, was,...","[in, what, us, state, did, kathmandu, first, e...","[[0.00080009917, -0.0023480232, -0.0011538627,...","[0.00022789818, -0.000552474, -6.692853e-05, 0...","[[0.00036538197, -0.0011221665, 0.00010991769,...","[-3.0412079e-05, -0.00010113185, -7.501468e-05..."
37594,Rangoon,414,What was Yangon previously known as?,"Kathmandu Metropolitan City (KMC), in order to...",Kathmandu,"[this, activity, has, been, further, enhanced,...","[what, was, yangon, previously, known, as]","[[-0.0004852763, -5.1644292e-05, -8.1352315e-0...","[2.2834672e-06, 4.3055956e-05, 2.4533767e-05, ...","[[-3.6202127e-05, -0.00038964432, 0.0004484944...","[-7.7654295e-05, 0.0002231892, -4.431147e-05, ..."
37595,Minsk,476,With what Belorussian city does Kathmandu have...,"Kathmandu Metropolitan City (KMC), in order to...",Kathmandu,"[this, activity, has, been, further, enhanced,...","[with, what, belorussian, city, does, kathmand...","[[0.0008059591, -0.0012151768, -0.0006767667, ...","[6.1412306e-05, -0.000257591, -0.00012535113, ...","[[-3.6202127e-05, -0.00038964432, 0.0004484944...","[-7.7654295e-05, 0.0002231892, -4.431147e-05, ..."
37596,1975,199,In what year did Kathmandu create its initial ...,"Kathmandu Metropolitan City (KMC), in order to...

In [20]:
print(scores)
print(scores['correct']/test_df.shape[0])

{'correct': 10003, 'incorrect': 27587}
0.26607969356812256


In [22]:
10003+27587

37590